# Metallicity Prior Generator
### Written by Noah Vowell
Updated by Jack Schulte on 9/25/23

In [1]:
import numpy as np 
import pandas as pd
import astropy.units as u
import astropy.constants as const 
from bs4 import BeautifulSoup as bs
import re
import requests
import scipy.stats as stats

In [2]:
def print_prior(TICID, sigma=1):
    '''
    Generates a metallicity prior from the mean and standard deviation of spectroscopic metallicities available on ExoFOP.

    TICID: The object's TESS Input Catalog identifier. Ex: for TIC 81247740, enter '81247740'
    sigma: The number of standard deviations that you would like as your prior width.
    '''

    url = 'https://exofop.ipac.caltech.edu/tess/target.php?id=' + TICID + '&json'
    page = requests.get(url)

    n_feh = page.json()['stellar_parameters'][4]['prov_num']

    feh = []
    for i in range(int(n_feh)):
        feh.append(float(page.json()['stellar_parameters'][5+i]['met']))
    feh = np.array(feh)

    print(f'{np.round(feh.mean(), 4)} +/- {np.round(sigma * np.std(feh),4)}')

In [4]:
print_prior('143526233', sigma=2)

0.2513 +/- 0.1557


In [4]:
def print_prior_fromfile(path, sigma=1):
    data = pd.read_csv(path, sep='\s+', header=0)
    for i in range(len(data)):
        if ('lsd' in data.method[i]) or ('Cl2.6' in data.method[i]):
            data = data.drop(i)

    print(f'{np.round(data.mh.mean(), 4)} +/- {np.round(sigma * np.std(data.mh),4)}')

In [11]:
path = '/Users/jack/Research/meep2/fits/toi3720/347051112.spc_table'
print_prior_fromfile(path, sigma=2)

0.245 +/- 0.3583


In [13]:
contam = 0.06116337
dilute_sigma = 0.1 * (contam / (1 + contam))

In [14]:
dilute_sigma

0.005763803362341842